# Load modules

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import json
import os
import re

In [2]:
df=pd.read_csv('/home/e077926/buscode_2023/05_E2_tata/data/cleaned_raw_bbox_df.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,doc,line_id,value,entity
0,X00016469612,1,TAN,O
1,X00016469612,1,WOON,O
2,X00016469612,1,YANN,O
3,X00016469612,2,BOOK,B-company
4,X00016469612,2,TA,I-company


# Create line dataset

In [3]:
df['sentence_id'] = df['doc'] + df['line_id'].astype(str)
df["sentence_id"] = LabelEncoder().fit_transform(df["sentence_id"] )

In [4]:
entity_list=df['entity'].unique().tolist()
new_entity_list=['O',
 'B-company',
 'I-company',
 'B-address',
 'I-address',
 'B-date',
 'I-date',
 'B-total']

In [5]:
ner_tag = list(range(0,8))

df['ner_tag']=df['entity'].replace(new_entity_list, ner_tag)
df.head()

,doc,line_id,value,entity,sentence_id,ner_tag
0,X00016469612,1,TAN,O,0,0
1,X00016469612,1,WOON,O,0,0
2,X00016469612,1,YANN,O,0,0
3,X00016469612,2,BOOK,B-company,11,1
4,X00016469612,2,TA,I-company,11,2


In [6]:
df_line=df.copy()

In [7]:
df_line['doc']=df_line['doc'].astype(str)
df_line['sentence_id']=df_line['sentence_id'].astype(int)
df_line['tokens']=df_line['value'].astype(str)
df_line['ner_tags']=df_line['ner_tag'].astype(int)
df_line['labels']=df_line['entity'].astype(str)

df_line=df_line[["sentence_id", "tokens", "labels", "ner_tags"]]

df_line.head()

,sentence_id,tokens,labels,ner_tags
0,0,TAN,O,0
1,0,WOON,O,0
2,0,YANN,O,0
3,11,BOOK,B-company,1
4,11,TA,I-company,2


In [8]:
# Réorganiser les données
grouped_df = df_line.groupby('sentence_id').agg({
    'tokens': list,
    'labels': list,
    'ner_tags': list,
}).reset_index()

# Renommer la colonne id
grouped_df.rename(columns={'id': 'sentence_id'}, inplace=True)
grouped_df.head()

,sentence_id,tokens,labels,ner_tags
0,0,"[TAN, WOON, YANN]","[O, O, O]","[0, 0, 0]"
1,1,"[25/12/2018, 8:13:39, PM]","[B-date, O, O]","[5, 0, 0]"
2,2,[CASHIER:],[O],[0]
3,3,[MANIS],[O],[0]
4,4,[MEMBER:],[O],[0]


In [9]:
grouped_df["sentence_id"] = LabelEncoder().fit_transform(grouped_df["sentence_id"] )

In [10]:
grouped_df.tail()

,sentence_id,tokens,labels,ner_tags
50171,50171,"[GOODS, SOLD, ARE, NOT, RETURNABLE, FOR, REFUN...","[O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
50172,50172,"[GST, REG, :, 001125220352]","[O, O, O, O]","[0, 0, 0, 0]"
50173,50173,"[TAX, INVOICE]","[O, O]","[0, 0]"
50174,50174,"[CB#, :, 87870]","[O, O, O]","[0, 0, 0]"
50175,50175,"[21/09/2017, 10:20:37, AM]","[B-date, O, O]","[5, 0, 0]"


In [11]:
grouped_df.to_csv('/home/e077926/buscode_2023/05_E2_tata/data/line_clean_bbox_df.csv')

In [1]:
grouped_df

NameError: name 'grouped_df' is not defined

# Create doc datset

In [24]:
# Réorganiser les données en groupant par 'doc' et 'line_id'
grouped_df_doc = df.groupby(['doc']).agg({
    'value': list,
    'entity': list,
    'ner_tag': list
}).reset_index()
# Renommer la colonne id
grouped_df_doc.rename(columns={'id': 'doc'}, inplace=True)
grouped_df_doc.head()


,doc,value,entity,ner_tag
0,X00016469612,"[TAN, WOON, YANN, BOOK, TA, .K(TAMAN, DAYA), S...","[O, O, O, B-company, I-company, O, I-company, ...","[0, 0, 0, 1, 2, 0, 2, 2, 0, 0, 3, 4, 4, 4, 4, ..."
1,X00016469619,"[TAN, WOON, YANN, INDAH, GIFT, &, HOME, DECO, ...","[O, O, O, B-company, I-company, I-company, I-c...","[0, 0, 0, 1, 2, 2, 2, 2, 0, 3, 4, 4, 4, 4, 0, ..."
2,X00016469620,"[TAN, WOON, YANN, MR, D.T.Y., (JOHOR), SDN, BH...","[O, O, O, B-company, O, I-company, I-company, ...","[0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0, 3, 4, 4, 4, ..."
3,X00016469622,"[TAN, WOON, YANN, YONGFATT, ENTERPRISE, (JM051...","[O, O, O, B-company, I-company, O, B-address, ...","[0, 0, 0, 1, 2, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, ..."
4,X00016469623,"[TAN, WOON, YANN, MR, D.I.Y., (M), SDN, BHD, (...","[O, O, O, B-company, I-company, I-company, I-c...","[0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 3, 4, 4, ..."


In [29]:
grouped_df_doc.to_csv('/home/e077926/buscode_2023/05_E2_tata/data/doc_clean_bbox_df.csv')

In [27]:
# Ajuster les options pour afficher plus de lignes
pd.set_option('display.max_rows', None)  # Avertissement : mettre à None peut surcharger votre système pour de très grands DataFrames

# Ajuster les options pour afficher plus de colonnes
pd.set_option('display.max_columns', None)

# Pour une meilleure lisibilité, vous pouvez également augmenter la largeur maximale des colonnes affichées
pd.set_option('display.max_colwidth', None)

In [28]:
grouped_df_doc

,doc,value,entity,ner_tag
0,X00016469612,"[TAN, WOON, YANN, BOOK, TA, .K(TAMAN, DAYA), SDN, BND, 789417-W, NO.53, 55,57, &, 59,, JALAN, SAGU, 18,, TAMAN, DAYA,, 81100, JOHOR, BAHRU,, JOHOR., DOCUMENT, NO, :, TD01167104, DATE:, 25/12/2018, 8:13:39, PM, CASHIER:, MANIS, MEMBER:, CASH, BILL, CODE/DESC, PRICE, DISC, AMOUNT, QTY, RM, RM, 9556939040116, KF, MODELLING, CLAY, KIDDY, FISH, 1, PC, *, 9.000, 0.00, 9.00, TOTAL:, ROUR, DING, ADJUSTMENT:, 0.00, ROUND, D, TOTAL, (RM):, 9.00, CASH, 10.00, CHANGE, 1.00, GOODS, SOLD, ARE, NOT, RETURNABLE, OR, EXCHANGEABLE, ***, ***, THANK, YOU, PLEASE, COME, AGAIN, !, 9.00]","[O, O, O, B-company, I-company, O, I-company, I-company, O, O, B-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, O, O, O, O, O, B-date, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-total, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 1, 2, 0, 2, 2, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,X00016469619,"[TAN, WOON, YANN, INDAH, GIFT, &, HOME, DECO, 27,JALAN, DEDAP, 13,, TAMAN, JAYA,, 81100, BAHRU,JOHOR., TEL:07-3507405, FAX:07-3558160, RECEIPT, 19/10/2018, 20:49:59, #01, CASHIER:, CN, LOCATION/SP:, 05, /0531, MB:, MO26588, ROOM, NO:, 01, 050100035279, DESC/ITEM, QTY, PRICE, AMT(RM), ST-PRIVILEGE, CARD/GD, INDAH, 88888, 1, 10.00, 10.00, GF-TABLE, LAMP/STITCH, <I>, 62483, 1, 55.90, 55.90, @DISC, 10.00%, -5.59, #TOTAL, QTY, 2, TOTAL, AMT................., RM, 60.31, ROUNDING, ADJ............, -0.01, RM, 60.30, CASH...................., RM, 70.30, CHANGE.................., RM, 10.00, THANK, YOU, PLEASE, COME, AGAIN, GOODS, SOLD, ARE, NOT, RETURNABLE, THANK, YOU, FLEASE, COME, AOSIN, GOODS, SOLD, ARE, NOT, RETURNABLE, DEALING, IN, WHOLESALE, AND, RETAIL., !, !, !, !]","[O, O, O, B-company, I-company, I-company, I-company, I-company, O, B-address, I-address, I-address, I-address, I-address, O, O, O, O, B-date, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-company, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-total, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 1, 2, 2, 2, 2, 0, 3, 4, 4, 4, 4, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,X00016469620,"[TAN, WOON, YANN, MR, D.T.Y., (JOHOR), SDN, BHD, (CO.REG, :, 933109-X), LOT, 1851-A, &, 1851-B,, JALAN, KPB, 6,, KAWASAN, PERINDUSTRIAN, BALAKONG,, 43300, SERI, KEMBANGAN,, SELANGOR, (MR, DIY, TESCO, TERBAU), -INVOICE-, CHOPPING, BOARD, 35.5X25.5CM, 803M#, EZ10HD05, -, 24, 8970669, 1, X, 19.00, 19.00, AIR, PRESSURE, SPRAYER, SX-575-1, 1.5L, HC03-7, -, 15, 9066468, 1, X, 8.02, 8.02, WAXCO, WINDSHILED, CLEANER, 120ML, WA14-3A, -, 48, 9557031100236, 1, X, 3.02, 3.02, BOPP, TAPE, 48MM*100M, CLEAR, FZ-04, -, 36, 6935818350846, 1, X, 3.88, 3.88, ITEM(S), :, 4, QTY(S), :, 4, TOTAL, RM, 33.92, ROUNDING, ADJUSTMENT, -RM, 0.02, TOTAL, ROUNDED, RM, 33.90, CASH, RM, 50.00, CHANGE, ...]","[O, O, O, B-company, O, I-company, I-company, I-company, O, O, O, B-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, I-address, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-total, O, O, O, O, ...]","[0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0, 3